In [1]:
#########################

TEST_MODE = False # needs to be correctly defined
#TEST_MODE = True
TEST_50 = False # needs to be correctly defined
#TEST_50 = True

TEST_TRAIN = 5000#0
#TEST_TRAIN = 1000000
TEST_VAL = 1000#0
#TEST_VAL = 100000

PART = 1   
VERSION = 4

VAL_TO_TEST = 1

#RESUMING = True
RESUMING = False

#FINAL_EPOCHS = 1 # only matters if PART=1 (train steps calculation)
FINAL_EPOCHS = 1.5

#TEST = True
TEST = False

LEARNING_CURVES_POINTS = 8

VALID_LOSS = False
#VALID_LOSS = True

TAW_CLIP = None
WEIGHT_DECAY = .01

#VALIDATION = True
VALIDATION = False
#########################

In [2]:
seeds = {
    1:1234,
    2:667,
    3:333,
    4:167,
    5:13,
    6:15,
    7:777,
    8:888,
}

SEED = seeds[VERSION]
SEED_VAL_SPLIT = SEED # different train/val but same pytorch

SEED

167

In [3]:
import numpy as np
import pandas as pd
import os
import sys

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings(action='once')

import shutil

import matplotlib.pyplot as plt

## PYTORCH GPT2 INITIALIZING

In [5]:
# installing Nvidia Apex without Internet access

! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a
from apex import amp

/opt/conda/lib/python3.6/site-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-t5i3hcg_
Created temporary directory: /tmp/pip-req-tracker-khmkzbbo
Created requirements tracker '/tmp/pip-req-tracker-khmkzbbo'
Created temporary directory: /tmp/pip-install-h4zc4sf7
Processing /kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
  Created temporary directory: /tmp/pip-req-build-bf65pmya
  Added file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a to build tracker '/tmp/pip-req-tracker-khmkzbbo'
    Running setup.py (path:/tmp/pip-req-build-bf65pmya/setup.py) egg_info for package from file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
    Running command python setup.py egg_info
    torch.__version__  =  1.1.0
    running egg_info
    creating pip-egg

In [6]:
# add the GPT2 pytorch repo to the PATH 

sys.path.insert(0, "../input/gpt2-pytorch/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master")
# using files from: https://github.com/pronkinnikita/pytorch-pretrained-BERT

# importing pytorch GPT2 functions

from pytorch_pretrained_bert import GPT2Tokenizer, GPT2ClassificationHeadModel
from pytorch_pretrained_bert import OpenAIAdam

In [7]:
GPT2_MODEL_PATH = '../input/gpt2-models'
DATA_DIR="../input/jigsaw-unintended-bias-in-toxicity-classification"
INPUT_DIR = "../input"
WORK_DIR = "../working"

In [8]:
# save a copy of gpt2_config in working dir so it appears as output with the model
shutil.copyfile(GPT2_MODEL_PATH + '/config.json', WORK_DIR + '/config.json')

'../working/config.json'

## PREPROCESSING

In [9]:
# converting the lines to GPT2 format

def convert_lines(example, max_seq_length, tokenizer):
    all_tokens = []
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)    
        if len(tokens_a) > MAX_SEQUENCE_LENGTH:
            tokens_a = tokens_a[-MAX_SEQUENCE_LENGTH:]
        one_token = tokenizer.convert_tokens_to_ids(tokens_a) + [0]*(MAX_SEQUENCE_LENGTH - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

In [10]:
# loading pytorch tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(GPT2_MODEL_PATH)

../input/gpt2-pytorch/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
../input/gpt2-pytorch/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]


In [11]:
MAX_SEQUENCE_LENGTH = 220

IDENTITY_COLUMNS = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

TARGET_COLUMN = 'target'

#Y_COLUMNS = ['target']
Y_COLUMNS = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [12]:
# loading train data

train_data = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_data = pd.read_csv(os.path.join(DATA_DIR, "test.csv")) # used for val size
len(train_data), len(test_data)

(1804874, 97320)

In [13]:
# PREPROCESSING #

# make sure all comment_text values are strings
train_data['comment_text'] = train_data['comment_text'].astype(str)

# filling nas
train_data = train_data.fillna(0)
# why zeros, it was applied after sequences filled 'comment_text' nans in original
# no nans in 'comment_text' so we can do it
# pd.read_csv(os.path.join(DATA_DIR, "train.csv"))['comment_text'].isnull().values.any() = False

# convert target to 0,1 if wanted
#train_data['target'] = (train_data['target']>=0.5).astype(float)

In [14]:
# train-val split
if VALIDATION:
    train_df, val_df = train_test_split(train_data, test_size=VAL_TO_TEST*len(test_data), random_state=SEED_VAL_SPLIT, shuffle=True, stratify=train_data[TARGET_COLUMN]>=.5)
else:
    train_df, val_df = train_test_split(train_data, test_size=VAL_TO_TEST*len(test_data), random_state=SEED_VAL_SPLIT, shuffle=True, stratify=train_data[TARGET_COLUMN]>=.5)
    train_df = pd.concat([train_df, val_df])
    
# sub-selecting train data
if PART % 2 != 0:
    train_df = train_df.head(int(len(train_df)/2))
else:
    train_df = train_df.tail(int(len(train_df)/2))
    
if TEST_MODE:
    train_df = train_df[:TEST_TRAIN]
    val_df = val_df[:TEST_VAL]
    print('TEST MODE\n\n')

if TEST_50:
    train_df = train_data.sample(TEST_TRAIN+TEST_VAL, random_state=SEED)
    val_df = train_df[TEST_TRAIN:TEST_TRAIN+TEST_VAL]
    train_df = train_df[:TEST_TRAIN]
    print('TEST MODE\n\n')
        
print('TRAINING:')
print(train_df['comment_text'].head())
if VALIDATION:
    print('\nVALIDATION:')
    print(val_df['comment_text'].head())

TRAINING:
1639862    We just need to make it clear that it's the de...
290043     trump has NO REGARD for native peoples.  He ha...
470464     Dion took the hit for Trudeau, who wanted the ...
29754      KK - No - Most arrivals in the past 30 years m...
322351     Given Trump takes over in January this is a wa...
Name: comment_text, dtype: object


In [15]:
# X, y
X = convert_lines(train_df["comment_text"], MAX_SEQUENCE_LENGTH, tokenizer) 
y = train_df[Y_COLUMNS].values

# X_val, y_val
if VALIDATION:
    X_val = convert_lines(val_df["comment_text"], MAX_SEQUENCE_LENGTH, tokenizer)
    y_val = val_df[Y_COLUMNS].values

# no need to fill 'comment_text' nas in train

In [16]:
#TAW_CLIP = 3
if TAW_CLIP is not None:
    train_df['ta_count'] = train_df['toxicity_annotator_count'] / train_df['toxicity_annotator_count'].mean()
    train_df['ta_count_coef'] = train_df['ta_count'].clip(upper=TAW_CLIP) / train_df['ta_count'].clip(upper=TAW_CLIP).mean()
    train_df['ta_count_coef'].describe()

In [17]:
# sample weights

for column in IDENTITY_COLUMNS + [TARGET_COLUMN]:
    train_df[column] = np.where(train_df[column] >= 0.5, True, False)
    
sample_weights = np.ones(len(train_df), dtype=np.float32)
sample_weights += train_df[IDENTITY_COLUMNS].sum(axis=1)
sample_weights += train_df[TARGET_COLUMN] * (~train_df[IDENTITY_COLUMNS]).sum(axis=1)
sample_weights += (~train_df[TARGET_COLUMN]) * train_df[IDENTITY_COLUMNS].sum(axis=1) * 5
if TAW_CLIP is not None:
    sample_weights *= train_df['ta_count_coef']
sample_weights /= sample_weights.mean()

sample_weights.describe()

count    902437.000000
mean          1.000000
std           1.402136
min           0.428977
25%           0.428977
50%           0.428977
75%           0.428977
max          21.019859
dtype: float64

In [18]:
##################
if VALIDATION:
    # sample weights valid
    val_df_boolean = val_df.copy() # we need val_df with continuous variables for correct validation

    for column in IDENTITY_COLUMNS + [TARGET_COLUMN]:
        val_df_boolean[column] = np.where(val_df_boolean[column] >= 0.5, True, False)

    sample_weights_valid = np.ones(len(val_df_boolean), dtype=np.float32)
    sample_weights_valid += val_df_boolean[IDENTITY_COLUMNS].sum(axis=1)
    sample_weights_valid += val_df_boolean[TARGET_COLUMN] * (~val_df_boolean[IDENTITY_COLUMNS]).sum(axis=1)
    sample_weights_valid += (~val_df_boolean[TARGET_COLUMN]) * val_df_boolean[IDENTITY_COLUMNS].sum(axis=1) * 5
    if TAW_CLIP is not None:
        sample_weights *= train_df['ta_count_coef']
    sample_weights_valid /= sample_weights_valid.mean()

    sample_weights_valid.describe()
##################

## TRAINING

In [19]:
OUTPUT_MODEL_FILE = "pytorch_model.bin"

LR = 2e-5
#BATCH_SIZE_TRAIN = 32 # OOM
BATCH_SIZE_TRAIN = 16
#BATCH_SIZE_VALID = 16 # OOM
BATCH_SIZE_VALID = 8
#ACCUMULATION_STEPS = 1
ACCUMULATION_STEPS = 2
#FINAL_EPOCHS = 1.5
FALSE_EPOCHS = 1
WARMUP = 0.05
#WEIGHT_DECAY = 0.01
#VALID_LOSS = False

DEVICE = torch.device('cuda')

#RESUMING = True

UTILS = ['gpt2-pytorch',
 'nvidiaapex',
 'jigsaw-unintended-bias-in-toxicity-classification',
 'gpt2-models']

if RESUMING:
    resuming_dir = [d for d in os.listdir('../input') if d not in UTILS][0]
    resuming_dir
    
#LEARNING_CURVES_POINTS = 25

In [20]:
# ensuring reproducibility

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [21]:
# defining train tensor

train_tensor = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.long),
                                       torch.tensor(y, dtype=torch.float),
                                       torch.tensor(sample_weights.values, dtype=torch.float),
                                       )
train_tensor

In [22]:
#################
if VALIDATION:
    # defining valid tensor

    valid_tensor = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.long),
                                           torch.tensor(y_val, dtype=torch.float),
                                           torch.tensor(sample_weights_valid.values, dtype=torch.float),
                                           )
    valid_tensor
#################

In [23]:
# loading the model architecture (& model weights if resuming)

if not RESUMING:
    model = GPT2ClassificationHeadModel.from_pretrained(GPT2_MODEL_PATH, clf_dropout=0.4, n_class=len(Y_COLUMNS))
    
else:
    model = GPT2ClassificationHeadModel.from_pretrained(os.path.join(INPUT_DIR, resuming_dir), clf_dropout=0.4, n_class=len(Y_COLUMNS))
    model.load_state_dict(torch.load(os.path.join(INPUT_DIR, resuming_dir, OUTPUT_MODEL_FILE))['state_dict'])

In [24]:
# zeroing model stored gradients
#model.zero_grad() 
# when starting, it doesnt make a difference as expected since training has yet to start
# when resuming, it doesnt make a difference either, optimizer.zero_grad() after last optimizer step probably did it already

# sending model to cuda
model = model.to(DEVICE)

In [25]:
# creating/loading optimizer

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

len_val = len(val_df) if VALIDATION else 0
if not (TEST_MODE or TEST_50):
    ###num_train_optimization_steps = int(FINAL_EPOCHS*len(train_data)/BATCH_SIZE_TRAIN/ACCUMULATION_STEPS) # bug
    num_train_optimization_steps = int(FINAL_EPOCHS*(len(train_data)-len_val)/BATCH_SIZE_TRAIN/ACCUMULATION_STEPS)
else:
    num_train_optimization_steps = int(FALSE_EPOCHS*len(train_tensor)/BATCH_SIZE_TRAIN/ACCUMULATION_STEPS)

optimizer = OpenAIAdam(optimizer_grouped_parameters, 
                     lr=LR,
                     warmup=WARMUP,
                     t_total=num_train_optimization_steps,
                     #schedule='warmup_cosine',
                    )

if RESUMING:
    optimizer.load_state_dict(torch.load(os.path.join(INPUT_DIR, resuming_dir, 'bert_pytorch.bin'))['optimizer'])
    # this will overwrite the optimizer
    
optimizer

OpenAIAdam (
Parameter Group 0
    b1: 0.9
    b2: 0.999
    e: 1e-08
    lr: 2e-05
    max_grad_norm: -1
    schedule: <pytorch_pretrained_bert.optimization.WarmupLinearSchedule object at 0x7f5055303358>
    vector_l2: False
    weight_decay: 0.01

Parameter Group 1
    b1: 0.9
    b2: 0.999
    e: 1e-08
    lr: 2e-05
    max_grad_norm: -1
    schedule: <pytorch_pretrained_bert.optimization.WarmupLinearSchedule object at 0x7f5055303358>
    vector_l2: False
    weight_decay: 0.0
)

In [26]:
BATCH_SIZE_VAL = 32 # only final validation
TOXICITY_COLUMN = 'target'
MODEL_NAME = 'model1' # allow us to store several validation data in same df

In [27]:
class JigsawEvaluator:

    def __init__(self, y_true, y_identity, power=-5, overall_model_weight=0.25):
        self.y = (y_true >= 0.5).astype(int)
        self.y_i = (y_identity >= 0.5).astype(int)
        self.n_subgroups = self.y_i.shape[1]
        self.power = power
        self.overall_model_weight = overall_model_weight

    @staticmethod
    def _compute_auc(y_true, y_pred):
        try:
            return roc_auc_score(y_true, y_pred)
        except ValueError:
            return np.nan

    def _compute_subgroup_auc(self, i, y_pred):
        mask = self.y_i[:, i] == 1
        return self._compute_auc(self.y[mask], y_pred[mask])

    def _compute_bpsn_auc(self, i, y_pred):
        mask = self.y_i[:, i] + self.y == 1
        return self._compute_auc(self.y[mask], y_pred[mask])

    def _compute_bnsp_auc(self, i, y_pred):
        mask = self.y_i[:, i] + self.y != 1
        return self._compute_auc(self.y[mask], y_pred[mask])

    def compute_bias_metrics_for_model(self, y_pred):
        records = np.zeros((3, self.n_subgroups))
        for i in range(self.n_subgroups):
            records[0, i] = self._compute_subgroup_auc(i, y_pred)
            records[1, i] = self._compute_bpsn_auc(i, y_pred)
            records[2, i] = self._compute_bnsp_auc(i, y_pred)
        return records

    def _calculate_overall_auc(self, y_pred):
        return roc_auc_score(self.y, y_pred)

    def _power_mean(self, array):
        total = sum(np.power(array, self.power))
        return np.power(total / len(array), 1 / self.power)

    def get_final_metric(self, y_pred):
        bias_metrics = self.compute_bias_metrics_for_model(y_pred)
        bias_score = np.average([
            self._power_mean(bias_metrics[0]),
            self._power_mean(bias_metrics[1]),
            self._power_mean(bias_metrics[2])
        ])
        overall_score = self.overall_model_weight * self._calculate_overall_auc(y_pred)
        bias_score = (1 - self.overall_model_weight) * bias_score
        return overall_score + bias_score

In [28]:
# TRAINING #


# initializing mixed precission
model, optimizer = amp.initialize(model, optimizer, opt_level="O1", verbosity=0)

# setting model in train mode
model = model.train()

# training loop
tq_epoch = tqdm(range(FALSE_EPOCHS))
for epoch in tq_epoch:
    
    train_loader = torch.utils.data.DataLoader(train_tensor, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
    ##################
    if VALID_LOSS:
        valid_loader = torch.utils.data.DataLoader(valid_tensor, batch_size=BATCH_SIZE_VALID, shuffle=False)
        lossf = 0
        valid_losses = []
        train_losses = []
        valid_scores = []
        avg_loss_valid = 0
        val_score = 0 ###
    else:
        lossf = None ####
    ##################
    avg_loss = 0.
    avg_accuracy = 0.

    tq_batch = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for i,(x_batch, y_batch, w_batch) in tq_batch:
        
        ##y_pred = model(x_batch.to(DEVICE), attention_mask=(x_batch>0).to(DEVICE), labels=None)
        y_pred = model(x_batch.to(DEVICE))
        
        #loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(DEVICE))
        loss = nn.BCEWithLogitsLoss(reduction='mean', weight=w_batch.unsqueeze(1).to(DEVICE))(y_pred,y_batch.to(DEVICE))

        # calculates gradients
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        
        # optimizer step
        if (i+1) % ACCUMULATION_STEPS == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()

        # tq_batch printing info
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item() # trying to smoothe learning curve?
        else:
            lossf = loss.item()
            
        if not VALID_LOSS:
            tq_batch.set_postfix(loss=lossf) ####
        
        # tq_epoch printing info
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(DEVICE)).to(torch.float)).item() / len(train_loader)
        
        ##################################
        if VALID_LOSS:
            if ((i+1) % int(len(train_loader)/LEARNING_CURVES_POINTS) == 0) or ((i+1) == len(train_loader)):

                model.eval()

                model_name = f'batch_{i+1}' ###
                avg_loss_valid = 0            
                tq_batch_valid = tqdm(enumerate(valid_loader), total=len(valid_loader), leave=False)
                val_preds = np.zeros((len(X_val))) ###
                for i,(x_batch_valid, y_batch_valid, w_batch_valid) in tq_batch_valid:
                    ##y_pred_valid = model(x_batch_valid.to(DEVICE), attention_mask=(x_batch_valid>0).to(DEVICE), labels=None)
                    y_pred_valid = model(x_batch_valid.to(DEVICE))

                    ##pred = model(x_batch.to(DEVICE), attention_mask=(x_batch > 0).to(DEVICE), labels=None) ###
                    val_preds[i * BATCH_SIZE_VALID:(i + 1) * BATCH_SIZE_VALID] = y_pred_valid[:, 0].detach().cpu().squeeze().numpy() ###

                    loss_batch_valid = nn.BCEWithLogitsLoss(reduction='mean', weight=w_batch_valid.unsqueeze(1).to(DEVICE))(y_pred_valid,y_batch_valid.to(DEVICE))                            
                    avg_loss_valid += loss_batch_valid.item() / len(valid_loader)

                val_pred = torch.sigmoid(torch.tensor(val_preds)).numpy()#.ravel() ###
                
                evaluator = JigsawEvaluator(val_df[TOXICITY_COLUMN].values, val_df[IDENTITY_COLUMNS].values)
                val_score = evaluator.get_final_metric(val_pred)
                #val_df[model_name] = val_pred ###            
                #bias_metrics_df = compute_bias_metrics_for_model(val_df, IDENTITY_COLUMNS, model_name, 'target') ###
                #val_score = get_final_metric(bias_metrics_df, calculate_overall_auc(val_df, model_name)) ###
                valid_scores.append(val_score) ###

                valid_losses.append(avg_loss_valid)

                model.train()

            train_losses.append(lossf) #?
            
            tq_batch.set_postfix(train_loss=lossf, valid_loss=avg_loss_valid, valid_score=val_score)
    if VALID_LOSS:
        tq_epoch.set_postfix(avg_loss=avg_loss, avg_accuracy=avg_accuracy, train_loss=lossf, valid_loss=avg_loss_valid, valid_score=val_score)
        ##################################                                   
    else:
        tq_epoch.set_postfix(avg_loss=avg_loss, avg_accuracy=avg_accuracy) ####


In [29]:
# saving model architecture and optimizer

state = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
torch.save(state, OUTPUT_MODEL_FILE)

In [30]:
# plotting learning curves

if VALID_LOSS:
    train_loss = train_losses
    valid_loss = [[int(len(train_loader)/LEARNING_CURVES_POINTS)*x for x in range(1, LEARNING_CURVES_POINTS+1)] + [len(train_loader)],
                  valid_losses,]
    valid_scoring = [[int(len(train_loader)/LEARNING_CURVES_POINTS)*x for x in range(1, LEARNING_CURVES_POINTS+1)] + [len(train_loader)],
                  valid_scores,] ###

    plt.figure(figsize=(17,17))
    plt.plot(train_loss, 'r-', linewidth=.1)
    plt.plot(valid_loss[0], valid_loss[1], 'b-', linewidth=3)
    #plt.plot(valid_scoring[0], valid_scoring[1], 'g-', linewidth=3) ###
    axes = plt.gca()
    axes.set_ylim([.175, .225])

In [31]:
#valid_loss[1]

In [32]:
#valid_scoring[1]

In [33]:
# saving plot data
if VALID_LOSS:
    np.save('train_loss', train_loss)
    np.save('valid_loss', valid_loss)
    np.save('valid_scoring', valid_scoring)

## VALIDATION

In [34]:
# FINAL VALIDATION #

if VALIDATION:
    # setting model in test mode
    for param in model.parameters():
        param.requires_grad=False
    model.eval()

    # initializing test_preds
    val_preds = np.zeros((len(X_val)))
    # defining valid tensor
    val_tensor = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.long))
    # defining valid loader
    val_loader = torch.utils.data.DataLoader(val_tensor, batch_size=BATCH_SIZE_VAL, shuffle=False)

    # testing loop
    for i, (x_batch,) in enumerate(tqdm(val_loader)):
        pred = model(x_batch.to(DEVICE), attention_mask=(x_batch > 0).to(DEVICE), labels=None)
        val_preds[i * BATCH_SIZE_VAL:(i + 1) * BATCH_SIZE_VAL] = pred[:, 0].detach().cpu().squeeze().numpy()

In [35]:
if VALIDATION:    
    val_pred = torch.sigmoid(torch.tensor(val_preds)).numpy()#.ravel()
    #val_df[MODEL_NAME] = val_pred
    #bias_metrics_df = compute_bias_metrics_for_model(val_df, IDENTITY_COLUMNS, MODEL_NAME, 'target')
    #bias_metrics_df
    evaluator = JigsawEvaluator(val_df[TOXICITY_COLUMN].values, val_df[IDENTITY_COLUMNS].values)
    val_score = evaluator.get_final_metric(val_pred)
    val_score

In [36]:
#get_final_metric(bias_metrics_df, calculate_overall_auc(val_df, MODEL_NAME))
# False/True instead of continuous variables in identity columns affect slightly metrics (this happened in 50, tests with weights)

## TEST

In [37]:
BATCH_SIZE_TEST = 32
#TEST = True

In [38]:
def test():
    
    # PREPROCESSING #

    # make sure all comment_text values are strings
    test_data['comment_text'] = test_data['comment_text'].astype(str)

    # filling nas & converting to BERT format
    X_test = convert_lines(test_data["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

    
    
    # TEST #

    # setting model in test mode already done by validation

    # initializing test_preds
    test_preds = np.zeros((len(X_test)))
    # defining test tensor
    test_tensor = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
    # defining test loader
    test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=BATCH_SIZE_TEST, shuffle=False)

    # testing loop
    for i, (x_batch,) in enumerate(tqdm(test_loader)):
        pred = model(x_batch.to(DEVICE), attention_mask=(x_batch > 0).to(DEVICE), labels=None)
        test_preds[i * BATCH_SIZE_TEST:(i + 1) * BATCH_SIZE_TEST] = pred[:, 0].detach().cpu().squeeze().numpy()

    
    
    # SUBMISSION #

    test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

    submission = pd.DataFrame.from_dict({
        'id': test_data['id'],
        'prediction': test_pred
    })
    submission.to_csv('submission.csv', index=False)

    return submission.head()

In [39]:
if TEST: test()